In [ ]:
import requests, bs4
from lxml import etree
import pandas as pd
import re
from tqdm import trange
import json

pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",100)

In [ ]:
def news_crawler(page,board='all'):
    news_url = f'https://news.ltn.com.tw/ajax/breakingnews/{board}/{page}'
    response = requests.get(news_url)
    news_content = response.json()

    try:
        news_ID=[]
        news_type=[]
        title=[]
        photo_L=[]
        urls=[]
        news_content_o=[]
        for i in news_content['data']:
            news_ID.append(i['no'])
            news_type.append(i['type_cn'])
            title.append(i['title'])
            photo_L.append(i['photo_L'])
            urls.append(i['url'])
            news_content_o.append(i['summary'])
    except:
        news_ID=[]
        news_type=[]
        title=[]
        photo_L=[]
        urls=[]
        news_content_o=[]
        for i in news_content['data'].values():
            news_ID.append(i['no'])
            news_type.append(i['type_cn'])
            title.append(i['title'])
            photo_L.append(i['photo_L'])
            urls.append(i['url'])
            news_content_o.append(i['summary'])


    time_frame=[]
    for i in urls:
        sub_response = requests.get(i)
        sub_content = sub_response.content.decode()
        html = etree.HTML(sub_content)
        time_frame.append(html.xpath("//span[@class='time']/text()"))

    time_n_s=[]
    for t in time_frame:
        time_n_s.append(t[0])
    time_s = [re.sub('\n', '', time_n_s[i]) for i in range(len(time_n_s))]
    time_o = [re.sub(' ', '', time_s[i]) for i in range(len(time_s))]

    m=[]
    for t in time_o: 
        m.append(re.findall(r'.{1}',t))
    m = pd.DataFrame(m)

    year = m[0]+m[1]+m[2]+m[3]
    month = m[5]+m[6]
    day = m[8]+m[9]
    YMD = m[0]+m[1]+m[2]+m[3]+'/'+m[5]+m[6]+'/'+m[8]+m[9]
    time = m[10]+m[11]+m[12]+m[13]+m[14]


    
    

    news_content = [re.sub('\n', '', news_content_o[i]) for i in range(len(news_content_o))]

    

    news_table = pd.DataFrame({'文章代碼':news_ID,
                               '類別':news_type,
                               '新聞標題':title,
                               '日期':YMD,
                               '時間':time,
                               '文章內容':news_content,
                               '新聞圖片':photo_L,
                               '連結':urls})
    return news_table


In [ ]:
df = pd.concat([news_crawler(i, board='novelty') for i in trange(1,11)]).reset_index(drop=True)
df.head(100)

In [ ]:
df.to_csv("/Users/syunhua/Desktop/000_prase_python/dpp_news_crawler.csv")